In [ ]:
import sys
# sys.path.append("../SpikeCounter")
import numpy as np
import skimage.io as skio
import scipy.ndimage as ndi
import scipy.signal as signal
import scipy.stats as stats
import scipy.fft as fft
from skimage.measure import label
import matplotlib.pyplot as plt
from tifffile import imsave
import scipy.ndimage as ndimage
import matplotlib.colors as colors
import matplotlib.patches as patches
import colorcet as cc
from cycler import cycler
import os
from scipy import interpolate, optimize
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import time
import mat73
import pandas as pd
import parse
from spikecounter.analysis import traces, images
from spikecounter.ui import visualize, HyperStackViewer
from spikecounter import utils
from skimage import draw, morphology, transform, filters

# plt.style.use("../bio_pubs_presentation.mplstyle")
ss = StandardScaler()

In [ ]:
rootdir = "E:/Cohen_Lab/Data/test_locus_finding"
filename = "091430E14_gcamp_fast"
crosstalk_channel = "None"

## Convert to tif, downsample and denoise

In [ ]:
!matlab -nojvm -nosplash -nodesktop -r "addpath(genpath('Y:\Lab\Computer Code\2020RigControl'));addpath(genpath('Y:\Lab\Computer Code\Image Processing'));batch_vm_to_tiff2('{rootdir}/{filename}','{rootdir}', 1, 1);"

In [ ]:
!python preprocess_stim_experiment2.py {rootdir} {filename} {crosstalk_channel} --n_pcs 30 --remove_from_start 0 --remove_from_end 0 --scale_factor 4 --skewness_threshold 0

## Load image

In [ ]:
def image_to_sta(raw_img, fs=1, mask=None, plot=False, \
                 savedir=None, prom_pct=90, sta_bounds="auto", \
                 exclude_pks = None, offset=0, full_output=False):
    """ Convert raw image into spike triggered average
    """
    if savedir is not None:
        os.makedirs(savedir, exist_ok=True)

    if plot:
        fig1, axes = plt.subplots(2,2, figsize=(10,10))
        axes = axes.ravel()

    # Generate mask if not provided
    mean_img = raw_img.mean(axis=0)
    if mask is None:
        mask = mean_img > np.percentile(mean_img, 80)
        kernel_size = int(mask.shape[0]/50)
        mask = morphology.binary_closing(mask, selem=np.ones((kernel_size, kernel_size)))
    _ = visualize.display_roi_overlay(mean_img, mask.astype(int), ax=axes[0])
    
    raw_trace = images.image_to_trace(raw_img, np.tile(mask, (raw_img.shape[0], 1, 1)))
    
    # convert to DF/F
    
    dFF_img = images.get_image_dFF(raw_img)
    
    if savedir:
        skio.imsave(os.path.join(savedir, "dFF.tif"), dFF_img)

    dFF_mean = images.image_to_trace(dFF_img, mask=np.tile(mask, (dFF_img.shape[0], 1, 1)))

    # Identify spikes
    ps = np.percentile(dFF_mean,[10,prom_pct])
    prominence = ps[1] - ps[0]
    pks, _ = signal.find_peaks(dFF_mean, prominence=prominence)
    
    if exclude_pks:
        keep = np.ones(len(pks))
        keep[exclude_pks] = 0
        pks = pks[keep]

    if plot:
        axes[1].plot(np.arange(dFF_img.shape[0])/fs, dFF_mean)
        axes[1].plot(pks/fs, dFF_mean[pks], "rx")
        axes[1].set_xlabel("Time (s)")
        axes[1].set_ylabel(r"$F/F_0$")
        tx = axes[1].twinx()
        tx.plot(np.arange(dFF_img.shape[0])/fs, raw_trace, color="C1")
        tx.set_ylabel("Mean counts")

    # Automatically determine bounds for spike-triggered average
    if sta_bounds == "auto":
        try:
        # Align all detected peaks of the full trace and take the mean
            aligned_traces = traces.align_fixed_offset(np.tile(dFF_mean, (len(pks), 1)), pks)
        except Exception as e:
            print(e)
            return raw_trace
        mean_trace = np.nanmean(aligned_traces, axis=0)
        # Smooth
        spl = interpolate.UnivariateSpline(np.arange(len(mean_trace)), \
                                np.nan_to_num(mean_trace, nan=np.nanmin(mean_trace)), s=0.001)
        smoothed = spl(np.arange(len(mean_trace)))
        smoothed_dfdt = spl.derivative()(np.arange(len(mean_trace)))   
        # Find the first minimum before and the first minimum after the real spike-triggered peak
        if plot:
            axes[3].plot(mean_trace)
            axes[3].plot(smoothed, color="C1")
            tx = axes[3].twinx()
            tx.plot(smoothed_dfdt, color="C2")
            tx.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        try:
            minima_left = np.argwhere((smoothed_dfdt < 0)).ravel()
            minima_right = np.argwhere((smoothed_dfdt > 0)).ravel()
            # calculate number of samples to take before and after the peak
            b1 = pks[-1] - minima_left[minima_left<pks[-1]][-1]+offset
            b2 = minima_right[minima_right > pks[-1]][0] - pks[-1]+offset
        except Exception:
            return mean_trace
        
        if plot:
            axes[3].plot([pks[-1]-b1, pks[-1], pks[-1]+b2], mean_trace[[pks[-1]-b1, pks[-1], pks[-1]+b2]], "rx")
        


    # Collect spike traces according to bounds
    spike_traces = np.nan*np.ones((len(pks), b1+b2))
    spike_images = np.nan*np.ones((len(pks), b1+b2, dFF_img.shape[1], dFF_img.shape[2]))
    print(b1,b2)
    for idx, pk in enumerate(pks):
        n_prepend = max(0, b1-pk)
        n_append = max(0, pk+b2-len(dFF_mean))

        mean_block = np.concatenate([np.ones(n_prepend)*np.nan, dFF_mean[max(0, pk-b1):min(len(dFF_mean), pk+b2)], np.ones(n_append)*np.nan])
        img_block = np.concatenate([np.ones((n_prepend, dFF_img.shape[1], dFF_img.shape[2]))*np.nan, \
                                    dFF_img[max(0, pk-b1):min(len(dFF_mean), pk+b2),:,:], np.ones((n_append, dFF_img.shape[1], dFF_img.shape[2]))*np.nan])
        
        spike_traces[idx,:] = mean_block
        spike_images[idx,:,:] = img_block
    
    sta_trace = np.nanmean(spike_traces, axis=0)
    sta = np.nanmean(spike_images, axis=0)

    if plot:
        axes[2].plot(np.arange(b1+b2)/fs, sta_trace)
        axes[2].set_xlabel("Time (s)")
        axes[2].set_ylabel(r"$F/F_0$")
        axes[2].set_title("STA taps: %d + %d" % (b1,b2))
        plt.tight_layout()
    if savedir:
        skio.imsave(os.path.join(savedir, "sta.tif"), sta)
        if plot:
            plt.savefig(os.path.join(savedir, "QA_plots.tif"))
    if full_output:
        return sta, spike_images
    return sta

In [ ]:
print(filename)
img, expt_data = images.load_image(rootdir, filename, subfolder="denoised")
print(img.shape)

In [ ]:
dt_dict, dt = utils.traces_to_dict(expt_data)

In [ ]:
sta, spike_images = image_to_sta(img, fs=1/dt, plot=True, prom_pct=90, \
                   savedir=os.path.join(rootdir, "analysis", filename), offset=120, full_output=True)

In [ ]:
utils.display_zstack(sta[:,:,:, np.newaxis], c=0, pct_cutoffs=[5,99])

In [ ]:
beta, smoothed_vid = images.spline_timing(sta, s=0.05, n_knots=15)

In [ ]:
os.makedirs(os.path.join(rootdir, "analysis", filename, "individual_spikes"), exist_ok=True)
skio.imsave(os.path.join(rootdir, "analysis", filename, "spline_smoothed_vid.tif"), smoothed_vid)
np.savez(os.path.join(rootdir, "analysis", "%s_snapt_denoised.npz") % filename,\
         beta=beta)

In [ ]:
amplitude = beta[2,:,:]
amplitude_nanr = np.copy(amplitude)
minval = np.nanmin(amplitude)
amplitude_nanr[np.isnan(amplitude)] = minval
thresh = np.percentile(amplitude_nanr,40)
# thresh = filters.threshold_otsu(amplitude_nanr)
mask = amplitude_nanr>thresh
mask = morphology.binary_opening(mask, selem=morphology.disk(2))
mask = morphology.binary_closing(mask, selem=morphology.disk(2))
plt.imshow(mask)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(4,4))
hm = beta[1,:,:]
hm_nan = np.copy(hm)
hm_nan[~mask] = np.nan
hm_nan = hm_nan-np.nanmin(hm_nan)
im = ax1.imshow(hm_nan, vmin=np.percentile(hm_nan[~np.isnan(hm_nan)],1), \
               vmax=np.percentile(hm_nan[~np.isnan(hm_nan)],99))
plt.colorbar(im, ax=ax1)

In [ ]:
%matplotlib notebook
plt.close("all")
idx = 4
beta = np.load(files[idx])["beta"]
print(beta.shape)
amplitude = beta[2,:,:]
amplitude[np.isnan(amplitude)] = 0
p5, p95 = np.percentile(amplitude,[5,95])
amplitude[amplitude > p95] = p95
amplitude[amplitude < p5] = p5
#         mask = np.ones_like(amplitude, dtype=bool)
#         mask = amplitude > np.percentile(amplitude, 80)
#         mask = morphology.binary_opening(mask, np.ones((3,3)))
#         mask = morphology.binary_dilation(mask, np.ones((20,20)))
#         hm_time = beta[1,:,:]/fs
#         hm_time[~mask] = np.nan
h = HyperStackViewer(amplitude, width=6, height=6)
mask = h.select_region_clicky()

In [ ]:
new_mask = h._points_to_mask(h.points, propagate_z=True).squeeze()

In [ ]:
%matplotlib inline
plt.close('all')
sigma = 1

amplitude[np.isnan(amplitude)] = 0
hm_time = beta[1,:,:]*dt
hm_time_old = np.copy(hm_time)
hm_time[~mask] = np.nan

pmin, pmax = np.percentile(hm_time[~np.isnan(hm_time)],[0.5, 95])
hm_time[hm_time < pmin] = pmin
hm_time[hm_time > pmax] = pmax
hm_time = hm_time-np.min(hm_time[~np.isnan(hm_time)])

hm_time_smoothed = np.copy(hm_time)
hm_time_smoothed[np.isnan(hm_time_smoothed)] = pmax
hm_time_smoothed = ndimage.gaussian_filter(hm_time_smoothed, sigma)
try:
    focus_mask = hm_time_smoothed < np.percentile(hm_time_smoothed, 10)
    labels, num = label(focus_mask, return_num=True)
    region_sizes = [np.sum(1/(hm_time_smoothed[labels==i]+0.01)) for i in np.arange(1, num+1)]
    single_region_mask = labels == (np.argmax(region_sizes)+1)
    masked_hm_time = np.ma.masked_array(hm_time_smoothed, ~single_region_mask)
    min_index = np.unravel_index(np.ma.argmin(masked_hm_time), hm_time.shape)
except Exception as e:
    print(e)
    min_index = [np.nan, np.nan]

fig1, axes = plt.subplots(1,2, figsize=(12,6))
axes = axes.ravel()
im = axes[0].imshow(hm_time*1000, alpha=0.3)
axes[0].imshow(np.ma.masked_array(focus_mask, ~focus_mask), alpha=0.3, cmap="Blues")
plt.colorbar(im, label="ms", aspect=15, shrink=0.75)
#         freq = n_peaks_by_embryo[e][mask_idx]/30
axes[1].imshow(hm_time_smoothed)
#         im = axes.contour(np.flipud(hm_time)*1000)
axes[1].plot(min_index[1], min_index[0], "wx")

### Draw mask targeting initiation site

In [ ]:
hm_time_smoothed.shape[0]*4+ offset[0]*2

In [ ]:
nrows = 2048
ncols = 2048
radius = 20

target_mask = np.zeros((nrows, ncols), dtype=bool)
offset = [(nrows - hm_time_smoothed.shape[0]*4)//2, \
          (ncols - hm_time_smoothed.shape[1]*4)//2]
rr, cc = draw.disk((offset[0] + min_index[0]*4, offset[1] + min_index[1]*4), radius)
target_mask[rr,cc] = True

fig1, ax1 = plt.subplots(figsize=(4,4))
ax1.imshow(target_mask)

In [ ]:
skio.imsave(os.path.join(rootdir, "analysis", filename, "%s_locus_target.tif" % filename), target_mask)
skio.imsave(os.path.join(rootdir, "Snaps", "%s_locus_target.tiff" % filename), target_mask)